In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'context2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5794981%2F9518352%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240930T181848Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D624359f55f4a5d566df49a7722ebabcdea028838f52db73a2606070c7c162b4d595fd1d857d8f514be351c0ee6c550ead6f062638ed30a08926ddad271b5176ddf26ebdc55817543d422e4d6722ac0b36374b9bd819b3516ec611557c5f146fb64a053c1ce1c88956495502aaa64735f5ce4e180f61d2dd7b6caaabc2f5297a5e46a08fb160679318c4ca3f0aa27f790c72610e205b682522593ce0039a5284c1ae7d8aa9224948e4ee1f2fb0f384056705d0f22e7ff9e4b3a1487e839fe26b6f48ede3367d80933991e3ba9f28e4032c07e8d19869879fb03fb52e104838a782cfeab144c923b968a355d855db935a25e95194ad329abe59462270416f948ec,attempt-930:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5794984%2F9518355%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240930T181848Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D002295d86aed4ffbbdd9992f9f2a6732d898d1a2ea07d760db36216b847e920ee918ce57b2cf37775db15eca23119bb55d976ebb456b728d1b9a3945e097dda59e44152c32c4490cb76400313a5e81e40094a966e05ab97980f3cf4aca8ed8ccc51e6e4226cee840f07ffa935819dd4d139a2c2115a488dd0940b43b0f943600f05c17bc9838e67ebad1c3b7b7799db69d25472d4b53b2b2aca23a683b468e8057c8bfdea7f438fb1587b9f4c4f5bec563d6d60575042d4330dbe653a1500d505e05d44618626ea5b25ec0d075f6228c1d77cb8e0589225ad898eed35ecc15c8bb53edcfdff11721c89b8bb600d51d4ac7d929d079e8aa175a24ab3494ff8659'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 82980 bytes downloaded
Downloaded and uncompressed: context2
[==================================================] 14779 bytes downloaded
Downloaded and uncompressed: attempt-930
Data source import complete.


In [2]:
print(type(data))

<class 'bytes'>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from datasets import load_dataset

In [ ]:
df = pd.read_csv('/kaggle/input/attempt-930/ESSA qna_csv.csv')

In [ ]:
# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into train and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Check the sizes of the splits
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
pdf_path = '/kaggle/input/context2/ESSA Chunk 1.pdf'
context_text = ""

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        context_text += page.extract_text() + "\n"

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    if answer_start >= len(inputs['input_ids'][0]) or answer_end > len(inputs['input_ids'][0]):
        return "No valid answer found within the context."

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples['input_text'],  # Ensure this matches your dataset's structure
        padding="max_length",
        truncation=True,
        max_length=512
    )

In [ ]:
# Example usage of the answer function
question = "What is Title I?"
print(answer_question(question, context_text))